In [1]:
# Imports
import numpy as np
import pandas as pd
import folium as fm # For the map, will only work in Jupyter since it needs a browser
from folium.plugins import FastMarkerCluster # In order to add map markers
from pyproj import Proj, transform
import matplotlib.pyplot as plt
import time

np.set_printoptions(threshold=np.inf, linewidth = 500, suppress=True)
pd.set_option('display.max_rows', 3000)
pd.set_option('display.max_columns', 3000)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# data cleaning

In [2]:
# load in data
df = pd.read_csv('parking-violations-geodata.csv', delimiter=',', encoding = 'latin-1', index_col='Unnamed: 0', 
                 dtype={'Zip Code' : int})

In [3]:
# check formatting
df.head(1)

,Summons Number,Violation Code,Violation Time,House Number,Street Name,City & State,Zip Code,Lat,Long
0,8725653684,71,0319P,235,Bedford Ave,"Brooklyn, NY",11211,40.716,-73.960


In [4]:
#Loads in our blank map, zoomed in on Greenpoint, BK
m = fm.Map([40.7305, -73.9515], zoom_start=13)

m

In [5]:
# map between folium color options and violation codes
v_types = pd.unique(df['Violation Code'])
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 
          'cadetblue','darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'lightgray']

cdict = {v : c for v, c in zip(v_types, colors)}

In [6]:
# clear map for successive runs
m = fm.Map([40.7255, -73.9465], zoom_start=14)

for ix, row in df.iterrows():
    # create an icon object
    point = fm.Icon(color=cdict[row['Violation Code']])#, icon_color='white', icon=str(row['Violation Code']))
    # create a marker at the proper location
    mrkr = fm.Marker(location=(row['Lat'], row['Long']), icon=point)
    # add it to the map
    mrkr.add_to(m)

In [7]:
m